# Exploring and clustering the neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes #uncomment if needed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes #uncomment if needed
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

#### Obtaining the data that is in the table of postal codes and transform the data into a pandas dataframe.
#### Processing the cells that have an assigned borough. Ignoring cells with a borough that is Not assigned.

In [2]:
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

df_postal = table[0]
df_postal.rename(columns={"Postcode":"PostalCode","Neighbourhood":"Neighborhood"}, inplace=True)
df_postal = df_postal[df_postal["Borough"]!="Not assigned"]
df_postal

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


#### More than one neighborhood can exist in one postal code area.
#### PostalCode which appears more than once will be combined into one row with the neighborhoods separated with a comma

In [3]:
df_postal = df_postal.groupby("PostalCode").agg(lambda x:", ".join(set(x)))
df_postal.reset_index(inplace = True)
df_postal

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"West Hill, Guildwood, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
df_postal.loc[df_postal["Neighborhood"] == "Not assigned", "Neighborhood"] = df_postal.loc[df_postal["Neighborhood"] == "Not assigned", "Borough"]
df_postal

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"West Hill, Guildwood, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


#### Print the number of rows of your dataframe

In [5]:
df_postal.shape

(103, 3)

#### Getting the latitude and the longitude coordinates of each neighborhood

In [6]:
df_coords = pd.read_csv("http://cocl.us/Geospatial_data")
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging those 2 dataframes on "PostalCode" column

In [7]:
df_coords.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
df_postal = pd.merge(df_postal, df_coords[["PostalCode","Latitude","Longitude"]], on="PostalCode")
df_postal

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Guildwood, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


#### Getting coordinates of Toronto

In [8]:
address = "Toronto, Ontario"
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#### Creating Toronto map by folium
#### Using circle marker to create series of circles corresponding to each postcode (by latitude and longitude) and add each of the pointer to the map

In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# markers
for lat, lng, borough, neighborhood in zip(df_postal['Latitude'], df_postal['Longitude'], df_postal['Borough'], df_postal['Neighborhood']):
    print(lat, lng)
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

43.806686299999996 -79.19435340000001
43.7845351 -79.16049709999999
43.7635726 -79.1887115
43.7709921 -79.21691740000001
43.773136 -79.23947609999999
43.7447342 -79.23947609999999
43.7279292 -79.26202940000002
43.711111700000004 -79.2845772
43.716316 -79.23947609999999
43.692657000000004 -79.2648481
43.7574096 -79.27330400000001
43.750071500000004 -79.2958491
43.7942003 -79.26202940000002
43.7816375 -79.3043021
43.8152522 -79.2845772
43.799525200000005 -79.3183887
43.836124700000006 -79.20563609999999
43.8037622 -79.3634517
43.7785175 -79.3465557
43.7869473 -79.385975
43.7574902 -79.37471409999999
43.789053 -79.40849279999999
43.7701199 -79.40849279999999
43.752758299999996 -79.4000493
43.7827364 -79.4422593
43.7532586 -79.3296565
43.745905799999996 -79.352188
43.72589970000001 -79.340923
43.7543283 -79.4422593
43.7679803 -79.48726190000001
43.737473200000004 -79.46476329999999
43.7390146 -79.5069436
43.7284964 -79.49569740000001
43.7616313 -79.52099940000001
43.725882299999995 -79.315

#### Taking borough which contains "Toronto" in their names

In [13]:
borough_toronto = df_postal[df_postal['Borough'].str.contains('Toronto')].reset_index(drop=True)
borough_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill, Forest Hill ...",43.686412,-79.400049


#### Creating Toronto borough map by folium
#### Using circle marker to create series of circles corresponding to each postcode (by latitude and longitude) and add each of the pointer to the map

In [14]:
map_toronto_borough = folium.Map(location=[latitude, longitude], zoom_start=10)
# markers
for lat, lng, borough, neighborhood in zip(borough_toronto['Latitude'], borough_toronto['Longitude'], borough_toronto['Borough'], borough_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_borough)  
    
map_toronto_borough